In [ ]:
from google.colab import files

kaggle_username = "your username"
kaggle_key = "api key"

!echo $kaggle_username | docker login kaggle.com -u $kaggle_username --password-stdin
!kaggle datasets download -d taeefnajib/used-car-price-prediction-dataset

!unzip used-car-price-prediction-dataset.zip



/bin/bash: line 1: docker: command not found
Dataset URL: https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
100% 109k/109k [00:00<00:00, 295kB/s]
100% 109k/109k [00:00<00:00, 295kB/s]
Archive:  used-car-price-prediction-dataset.zip
  inflating: used_cars.csv           


In [1]:
import pandas as pd

In [2]:
db=pd.read_csv("used_cars.csv")

In [3]:


def normalize_data(df):
  # Normalize model_year
  df['model_year'] = (df['model_year'] - df['model_year'].min()) / (df['model_year'].max() - df['model_year'].min())

  # Normalize mileage
  df['milage'] = df['milage'].str.replace(' mi.', '', regex=False)
    # Remove commas
  df['milage'] = df['milage'].str.replace(',', '', regex=False)
  df['milage'] = df['milage'].astype(int)
  df['milage'] = (df['milage'] - df['milage'].min()) / (df['milage'].max() - df['milage'].min())

  # Fill missing fuel type with "gasoline"
  df['fuel_type'].fillna('gasoline', inplace=True)

  # Convert accident to 0 (None reported) or 1 (Accident reported)
  df['accident'].fillna('None reported', inplace=True)
  df['accident'] = (df['accident'] == 'None reported').astype(int)

  # Convert clean_title to 1 (Yes) or 0 (No)
  df['clean_title'] = (df['clean_title'] == 'yes').astype(int)

  df['price'] = df['price'].str.replace('$', '', regex=False)
  df['price'] = df['price'].str.replace(',', '', regex=False)
  df['price'] = df['price'].astype(int)



  return df

df = normalize_data(db.copy())

# Print the normalized data
print(df.head())


      brand                            model  model_year    milage  \
0      Ford  Utility Police Interceptor Base        0.78  0.125710   
1   Hyundai                     Palisade SEL        0.94  0.085557   
2     Lexus                    RX 350 RX 350        0.96  0.055006   
3  INFINITI                 Q50 Hybrid Sport        0.82  0.219313   
4      Audi        Q3 45 S line Premium Plus        0.94  0.024043   

       fuel_type                                             engine  \
0  E85 Flex Fuel  300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...   
1       Gasoline                               3.8L V6 24V GDI DOHC   
2       Gasoline                                     3.5 Liter DOHC   
3         Hybrid  354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...   
4       Gasoline                         2.0L I4 16V GDI DOHC Turbo   

        transmission                 ext_col int_col  accident  clean_title  \
0        6-Speed A/T                   Black   Black         0           

In [4]:
df = df.replace('–', pd.NA)
df['engine'].fillna('Unknown', inplace=True)
df['transmission'].fillna('Unknown', inplace=True)

In [5]:
categorical_cols = ['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col']
df = pd.get_dummies(df, columns=categorical_cols)

In [6]:
from sklearn.preprocessing import StandardScaler
import numpy as np
scaler1 = StandardScaler()
scaler1.fit(df[['price']])

StandardScaler()

In [7]:
df

,model_year,milage,accident,clean_title,price,brand_Acura,brand_Alfa,brand_Aston,brand_Audi,brand_BMW,...,int_col_Titan Black,int_col_Titan Black / Quarzit,int_col_Tupelo,int_col_Very Light Cashmere,int_col_WHITE,int_col_Walnut,int_col_Whisper Beige,int_col_White,int_col_White / Brown,int_col_Yellow
0,0.78,0.125710,0,0,10300,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0.94,0.085557,0,0,38005,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0.96,0.055006,1,0,54598,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0.82,0.219313,1,0,15500,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,0.94,0.024043,1,0,34999,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004,0.98,0.001516,1,0,349950,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4005,0.96,0.026673,1,0,53900,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4006,0.96,0.004979,1,0,90998,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4007,0.92,0.081255,1,0,62999,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
rmse_standardized=-0.435233

rmse_original = scaler1.inverse_transform([[rmse_standardized]])[0][0]
rmse_original

10299.997114065169

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_cols = ['model_year', 'milage', 'accident', 'clean_title']
# numerical_cols = ['model_year', 'milage', 'accident', 'clean_title', 'price']

df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [10]:
# prompt: train test split and run xgboost with varying params and return best rmse

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    'colsample_bytree': 0.3,
    'learning_rate': 0.3,
    'max_depth': 5,
    'alpha': 10
}

model = xgb.XGBRegressor(**params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("RMSE:", rmse)

# Try different parameters and find the best RMSE
best_rmse = rmse
best_params = params

for colsample_bytree in [0.2, 0.3, 0.4,0.5]:
  for learning_rate in [0.2, 0.3,0.4]:
    for max_depth in [4, 5, 6,7]:
      for alpha in [10, 20,30]:
        params = {
            'colsample_bytree': colsample_bytree,
            'learning_rate': learning_rate,
            'max_depth': max_depth,
            'alpha': alpha
        }

        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))

        if rmse < best_rmse:
          best_rmse = rmse
          best_params = params

print("Best RMSE:", best_rmse)
print("Best parameters:", best_params)


RMSE: 136019.70014752858


In [ ]:
X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

# Convert the datasets into DMatrix, which is a data structure that XGBoost uses
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set parameters for XGBoost
params = {'colsample_bytree': 0.3, 'learning_rate': 0.3, 'max_depth': 6, 'alpha': 10}

# Train the model
model = xgb.train(params, dtrain, num_boost_round=100)

# Make predictions
y_pred = model.predict(dtest)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")


RMSE: 135488.64102120302


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np


model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)


rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")
